### Import Modules

In [19]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [88]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [89]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'chained' : False,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
}

In [90]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], F2=p_dict['F2'], include_FV=(p_dict['order']['vol']>0), 
                                 include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(
    fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'], 
    include_su2_isospin_corrrection=p_dict['include_su2_isospin_corrrection'],
    fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct']
)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

D:\Documents\School Work\Research\fk-fpi\priors\ma.csv
Making fits...
Done!
100% complete Time (s):  2.47599983215
Compiling results...

Model: ma_FpiFpi_nnlo_FV_alphaS_logSq

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1893(31) [1.1956(19)]   (delta_su2 = -0.00406(69))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
   A_loga 0               0.50 (73)            1               0.0 (1.6)
          1               0.00 (87)        A_a 0               0.70 (51)
      L_5 0           -0.00030 (33)            1               0.00 (87)
          1             0.0000 (50)        A_p 0               1.90 (15)
      A_k 0              -0.3 (1.2)            1               0.0 (1.6)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1.3 [16]    Q = 0.18    logGBF = 6

In [71]:
def test(self):
    w0_dict = {
        'a15' : gv.gvar('0.1511(18)'),
        'a12' : gv.gvar('0.1216(11)'),
        'a09' : gv.gvar('0.0873(11)')
    }
    
    gv_data = {}
    
    F0 = 131.5 / np.sqrt(2)
    hbar_c = 197.327
    
    abbrs = self.abbrs
    for abbr in abbrs:
        gv_data[abbr] ={}
        gv_data[abbr]['lam2_chi'] = (4 * np.pi *w0_dict[abbr[:3]] *F0 / hbar_c)**2
        print abbr, gv_data[abbr]['lam2_chi']

test(bootstrapper)

a09m220 0.2672(67)
a09m310 0.2672(67)
a09m350 0.2672(67)
a09m400 0.2672(67)
a12m130 0.5185(94)
a12m220 0.5185(94)
a12m220L 0.5185(94)
a12m220S 0.5185(94)
a12m310 0.5185(94)
a12m350 0.5185(94)
a12m400 0.5185(94)
a15m135XL 0.801(19)
a15m220 0.801(19)
a15m310 0.801(19)
a15m350 0.801(19)
a15m400 0.801(19)


In [68]:
bootstrapper.fit_data['a09m220']['lam2_chi']

0.2899(11)

In [86]:
{abbr : bootstrapper.fit_data[abbr]['lam2_chi'] for abbr in bootstrapper.fit_data.keys()}

{u'a09m220': 0.2899(11),
 u'a09m310': 0.32727(98),
 u'a09m350': 0.3434(12),
 u'a09m400': 0.3694(11),
 u'a12m130': 0.5118(25),
 u'a12m220': 0.5441(24),
 u'a12m220L': 0.5463(24),
 u'a12m220S': 0.5434(29),
 u'a12m310': 0.5950(21),
 u'a12m350': 0.6266(29),
 u'a12m400': 0.6669(23),
 u'a15m135XL': 0.8069(37),
 u'a15m220': 0.8362(19),
 u'a15m310': 0.8954(21),
 u'a15m350': 0.9339(27),
 'a15m400': 0.9950(31)}

In [61]:
'a09m220'[:3]

'a09'

In [57]:
hbar_c = 197.327

fit_data['a09m220']['mpi'][0]* hbar_c / (0.09 )

214.64777129873062